In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
I Modèle avec variables de type Features

In [51]:
# Import des .pkl
# X_train = pd.read_pickle("../data/X_train_n.pkl")
# Y_train = pd.read_pickle("../data/Y_train_n.pkl")
# X_test = pd.read_pickle("../data/X_test_n.pkl")
# Y_test = pd.read_pickle("../data/Y_test_n.pkl")

In [14]:
df_for_cv_interaction_and_dummies = pd.read_pickle("df_for_cv_interaction_and_dummies.pkl")
all_var_x = [each for each in list(df_for_cv_interaction_and_dummies) if each != "count" and each != "datetime"]
X_train = df_for_cv_interaction_and_dummies[all_var_x]
Y_train = df_for_cv_interaction_and_dummies[["count"]]

In [15]:
# Calcul des Y = log(Y + 1)
Y_train['log_count'] = Y_train['count'].map(lambda x: np.log1p(x))
# Y_test['log_count'] = Y_test['count'].map(lambda x: np.log1p(x))
Y_train = Y_train.drop('count', axis = 1)
# Y_test = Y_test.drop('count', axis = 1)

/home/osboxes/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
# Hyperparamètre de régularisation
# alpha = 0.01 to 100 ?

In [18]:
import time
now = time.time()
model = Ridge()
parameters = {'alpha':[0.1,0.5,1,5]}
grid_ridge = GridSearchCV(model,parameters, cv=5)
grid_ridge.fit(X_train, Y_train)
end = time.time()
print (end-now)

1027.274558544159


In [25]:
pd.DataFrame(grid_ridge.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,46.564941,2.426892,0.137703,0.005383,0.1,{'alpha': 0.1},0.663371,0.857091,0.852743,0.875088,0.847362,0.819117,0.078445,4
1,48.039030,3.518372,0.144442,0.004634,0.5,{'alpha': 0.5},0.740560,0.879967,0.870080,0.897006,0.888853,0.855283,0.058072,3
2,48.417894,2.248387,0.148158,0.002556,1,{'alpha': 1},0.776712,0.891219,0.878923,0.904294,0.904951,0.871211,0.048220,2
3,51.248157,5.419785,0.155741,0.005687,5,{'alpha': 5},0.841465,0.914510,0.901532,0.914590,0.934707,0.901355,0.031776,1


In [47]:
def RMLSE(pred,Y):
    # resultat = np.sqrt(np.mean((grid.predict(X) - Y.values)**2, axis=0))
    # resultat = np.sqrt(np.mean((pred - Y.values)**2, axis=0))
    resultat = np.sqrt(np.mean((pred - Y)**2, axis=0))
    return resultat

In [53]:
%%time

from sklearn.metrics import make_scorer

rmsle = make_scorer(RMLSE, greater_is_better=False)

model = Ridge()
# La valeur alpha = 5 semblait donner le meilleur modèle
# parameters = {'alpha':[0.1,0.5,1,5]}
# parameters = {'alpha':[1, 5, 10, 15]}
parameters = {'alpha':[15, 25, 50, 100]}
# grid_ridge = GridSearchCV(model,parameters, cv=5, scoring = "neg_mean_squared_log_error")
# grid_ridge.fit(X_train, Y_train)
grid_ridge = GridSearchCV(model,parameters, cv=5, scoring=rmsle, n_jobs = 2, return_train_score = True)
grid_ridge.fit(X_train, Y_train)

CPU times: user 1min 38s, sys: 2.52 s, total: 1min 41s
Wall time: 19min 16s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=2, param_grid={'alpha': [15, 25, 50, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=make_scorer(RMLSE, greater_is_better=False), verbose=0)

In [30]:
pd.DataFrame(grid_ridge.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,50.719912,3.121417,0.165485,0.030068,0.1,{'alpha': 0.1},-0.034452,-0.014439,-0.017033,-0.011893,-0.012700,-0.018105,0.008363,4
1,50.297228,1.488643,0.157237,0.011190,0.5,{'alpha': 0.5},-0.028627,-0.012281,-0.015171,-0.010058,-0.009637,-0.015156,0.007018,3
2,47.793118,0.527379,0.148534,0.007421,1,{'alpha': 1},-0.025730,-0.010999,-0.014187,-0.009480,-0.008395,-0.013759,0.006297,2
3,48.133455,0.410139,0.148430,0.004248,5,{'alpha': 5},-0.020167,-0.008149,-0.011630,-0.008803,-0.006242,-0.010999,0.004900,1


In [52]:
# Avec : parameters = {'alpha':[1, 5, 10, 15]}
pd.DataFrame(grid_ridge.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,54.198833,2.107559,0.167053,0.031623,1,{'alpha': 1},-0.645110,-0.432378,-0.481538,-0.431428,...,-0.481857,0.084420,4,-0.224860,-0.239940,-0.235906,-0.240907,-0.241911,-0.236705,0.006264
1,54.704508,2.829823,0.160618,0.020526,5,{'alpha': 5},-0.543580,-0.383305,-0.434257,-0.407561,...,-0.423166,0.066708,3,-0.229373,-0.244364,-0.240696,-0.245573,-0.246859,-0.241373,0.006343
2,57.379488,3.427663,0.163011,0.006985,10,{'alpha': 10},-0.516910,-0.366144,-0.416298,-0.402085,...,-0.406013,0.063261,2,-0.233900,-0.248817,-0.245492,-0.250243,-0.251559,-0.246002,0.006380
3,58.637168,4.031780,0.157926,0.002475,15,{'alpha': 15},-0.506536,-0.358829,-0.408961,-0.400449,...,-0.399773,0.061479,1,-0.238247,-0.253132,-0.249924,-0.254637,-0.255914,-0.250370,0.006383


In [54]:
pd.DataFrame(grid_ridge.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,104.071278,4.602311,0.199423,0.043426,15,{'alpha': 15},-0.506536,-0.358829,-0.408961,-0.400449,...,-0.399773,0.061479,2,-0.238247,-0.253132,-0.249924,-0.254637,-0.255914,-0.250370,0.006383
1,103.017336,4.135794,0.193576,0.009865,25,{'alpha': 25},-0.498350,-0.354368,-0.404909,-0.401642,...,-0.396964,0.058801,1,-0.246784,-0.261598,-0.258293,-0.263095,-0.264232,-0.258800,0.006331
2,109.901650,3.348438,0.180359,0.003729,50,{'alpha': 50},-0.495119,-0.360822,-0.411739,-0.413347,...,-0.404196,0.053710,3,-0.266877,-0.281327,-0.277213,-0.282553,-0.283415,-0.278277,0.006084
3,114.419825,4.654415,0.207904,0.053241,100,{'alpha': 100},-0.501128,-0.384872,-0.434737,-0.441195,...,-0.426063,0.046837,4,-0.300092,-0.313435,-0.307640,-0.314142,-0.314998,-0.310061,0.005615


On soumet le modèle avec alpha = 25 qui semble etre le meilleur modèle

In [60]:
# C'est le RMLSE sur "train" de la cross validation
result_model = pd.DataFrame(grid_ridge.cv_results_)
result_model[result_model["rank_test_score"] == 1][["mean_train_score"]]

,mean_train_score
1,-0.2588


In [61]:
# C'est le RMLSE sur "test" de la cross validation
grid_ridge.best_score_ 
# Avec la métrique interne à Scikit learn, on avait : -0.010999027326119631, soit ~ 0.011 pour alpha = 5
# Cette fois avec alpha = 25: -0.39696411418593996

-0.39696411418593996

In [66]:
df_to_predict = pd.read_pickle("../data/df_test_kaggle_interaction_and_dummies.pkl") 
df_to_predict.shape
df_to_predict.head()

df_temp = df_to_predict.drop("datetime", axis = 1)

df_to_predict["count"] = list((np.exp(grid_ridge.predict(df_temp)) - 1)[:,0])

In [68]:
df_to_predict[["count"]].head()

,count
0,10.987431
1,5.739557
2,3.279754
3,2.806729
4,3.335188


In [69]:
df_to_predict[["datetime", "count"]].to_csv("../data/submission_to_send/pred_linear_ridge_regression_correct_cv.csv", index= False)

In [ ]:
# ROOT MEAN SQUARED LOGARITHMIC ERROR : privilégier des prédictions au-dessous de la vérité

Linear regression + New : ON N'A PAS FAIT les features polynomiaux sur les var numériques